## Exercise 1: Prompt Engineering

Let's consider LLAMA as our starting point. In the following, we see a typical prompt feeding and text generation with LLAMA

In [83]:
from huggingface_hub import login

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


model_id = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Assuming model and tokenizer are already loaded
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Move model to the device (GPU if available)
model = model.to(device)


query = "Tell me the capital of France."

prompt = f"Question: {query}\n\nAnswer:"  # Create the prompt for LLAMA (context + query)

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

# Generate a response
output = model.generate(
    inputs['input_ids'],  # Tokenized input
    max_length=100,         # Limit response length to avoid extra text
    temperature=0.7,        # Lower temperature to reduce randomness
    do_sample=True,        # Disable sampling for deterministic output
    pad_token_id=tokenizer.eos_token_id  # Ensure the model doesn't go beyond the end token

)

# Decode the response into human-readable text
response = tokenizer.decode(output[0], skip_special_tokens=True).strip()

print("Response:", response)



cuda
Response: Question: Tell me the capital of France.

Answer: **A. PARIS**


### Fitz

Reference libraries to install: pip install openai pymupdf faiss-cpu scikit-learn

PyMuPDF is a Python library that provides tools for working with PDF files (as well as other document formats like XPS, OpenXPS, CBZ, EPUB, and FB2). It's built on the MuPDF library, a lightweight, high-performance PDF and XPS rendering engine. With PyMuPDF, you can perform various tasks like reading, creating, editing, and extracting content from PDFs, images, and annotations.

In [84]:
import fitz

#open an example pdf
doc = fitz.open("example2.pdf")

# Extract text from the first page
page = doc.load_page(0)
text = page.get_text("text")  # Use 'text' mode to get raw text
print(text)


Associazione Calcio Milan, commonly referred to as AC Milan or simply Milan, is an Italian 
professional football club based in Milan, Lombardy. Founded in 1899, the club competes in the Serie 
A, the top tier of Italian football. In its early history, Milan played its home games in different grounds 
around the city before moving to its current stadium, the San Siro, in 1926. The stadium, which was 
built by Milan's second chairman, Piero Pirelli and has been shared with Inter Milan since 1947, is the 
largest in Italian football, with a total capacity of 75,817. The club has a long-standing rivalry with Inter, 
with whom they contest the Derby della Madonnina, one of the most followed derbies in football. 
 
Milan has spent its entire history in Serie A with the exception of the 1980–81 and 1982–83 seasons. 
Silvio Berlusconi’s 31-year tenure as Milan president was a standout period in the club's history, as 
they established themselves as one of Europe's most dominant and successful

### Example: Text Summarization

Let's ask LLAMA to perform a summarization of the example PDF.

In [7]:
def read_pdf(doc):
    full_text= ""
    for num in range(len(doc)):
        page = doc.load_page(num)
        full_text += page.get_text('text')
    return full_text

In [4]:
from transformers import pipeline

In [85]:
#define the prompt to ask for text summarization. 
text_summarization_prompt = " Write a concise summary of the main ideas in article below. "      #define your prompt here
text = read_pdf(doc=doc)                           #load here the FULL text of the article
p1 =  """Question: {PROMPT}. article: {BODY} \nAnswer:""".format(PROMPT=text_summarization_prompt, BODY=text)

#feed the prompt to llama
#print the result of text summarization into bullets
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer(p1, return_tensors="pt", padding=True, truncation=True).to(device)



# Generate a response
output = model.generate(
    inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_new_tokens=300,  # Limit output to a reasonable size (adjust as needed)
    top_k=10,  # Sampling parameters
    num_return_sequences=1,  # Generate a single response
    temperature=0.7,  # Adjust randomness
    repetition_penalty=1.0,  # Reduce repetition
    length_penalty=1.0,  # Penalize longer responses
    eos_token_id=tokenizer.eos_token_id,  # End generation at EOS token
    pad_token_id=tokenizer.pad_token_id,  # Avoid padding tokens
    early_stopping=True  # Stop once the model generates the response
)

# Decode the response into human-readable text
r1 = tokenizer.decode(output[0], skip_special_tokens=True).strip()

print(r1)

Question:  Write a concise summary of the main ideas in article below.. article: Associazione Calcio Milan, commonly referred to as AC Milan or simply Milan, is an Italian 
professional football club based in Milan, Lombardy. Founded in 1899, the club competes in the Serie 
A, the top tier of Italian football. In its early history, Milan played its home games in different grounds 
around the city before moving to its current stadium, the San Siro, in 1926. The stadium, which was 
built by Milan's second chairman, Piero Pirelli and has been shared with Inter Milan since 1947, is the 
largest in Italian football, with a total capacity of 75,817. The club has a long-standing rivalry with Inter, 
with whom they contest the Derby della Madonnina, one of the most followed derbies in football. 
 
Milan has spent its entire history in Serie A with the exception of the 1980–81 and 1982–83 seasons. 
Silvio Berlusconi’s 31-year tenure as Milan president was a standout period in the club's history

### Adding a System Prompt

Llama was trained with a system message that set the context and persona to assume when solving a task. One of the unsung advantages of open-access models is that you have full control over the system prompt in chat applications. This is essential to specify the behavior of your chat assistant –and even imbue it with some personality–, but it's unreachable in models served behind APIs.


In [26]:
#default standard system message from the Hugging Face blog to the prompt from above
system_prompt = "<<SYS>> You are a helpful, respectful and honest assistant. \
    Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, \
    unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses \
    are socially unbiased and positive in nature. If a question does not make any sense, or is not factually \
    coherent, explain why instead of answering something not correct. If you don't know the answer to a question, \
    please don't share false information. <</SYS>>"

#concatenate the system prompt with your pront and get the response
p2 =  """{SYS}\nQuestion: {PROMPT}. article: {BODY} \nAnswer:""".format(SYS=system_prompt, PROMPT=text_summarization_prompt, BODY=text)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer(p2, return_tensors="pt", padding=True, truncation=True).to(device)



# Generate a response
output = model.generate(
    inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_new_tokens=300,  # Limit output to a reasonable size (adjust as needed)
    top_k=10,  # Sampling parameters
    num_return_sequences=1,  # Generate a single response
    temperature=0.7,  # Adjust randomness
    repetition_penalty=1.0,  # Reduce repetition
    length_penalty=1.0,  # Penalize longer responses
    eos_token_id=tokenizer.eos_token_id,  # End generation at EOS token
    pad_token_id=tokenizer.pad_token_id,  # Avoid padding tokens
    early_stopping=True  # Stop once the model generates the response
)

# Decode the response into human-readable text
r2 = tokenizer.decode(output[0], skip_special_tokens=True).strip()

print(r2)

#what changes?

<<SYS>> You are a helpful, respectful and honest assistant.     Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,     unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses     are socially unbiased and positive in nature. If a question does not make any sense, or is not factually     coherent, explain why instead of answering something not correct. If you don't know the answer to a question,     please don't share false information. <</SYS>>
Question:  Write a concise summary of the main ideas in article below.. article: Associazione Calcio Milan, commonly referred to as AC Milan or simply Milan, is an Italian 
professional football club based in Milan, Lombardy. Founded in 1899, the club competes in the Serie 
A, the top tier of Italian football. In its early history, Milan played its home games in different grounds 
around the city before moving to its current stadium, the San Siro, in 1

### Customizing the System prompt

With Llama we have full control over the system prompt. The following experiment will instruct Llama to assume the persona of a researcher tasked with writing a concise brief.

Apply the following changes the original system prompt:
- Use the researcher persona and specify the tasks to summarize articles.
- Remove safety instructions; they are unnecessary since we ask Llama to be truthful to the article.


In [ ]:
new_system_prompt = """<<SYS>> You are a helpful assistant. 
    When summarizing an article or providing key points, you must:
    1. Use **bullet points** to list the main ideas.
    2. **Bullet points must be formatted like this:**
       - First key point goes here.
       - Second key point goes here.
       - Third key point goes here.
    3. Each bullet point should be one short sentence.
    4. Keep bullet points clear, concise, and easy to read.
    5. Make sure every distinct idea from the article is listed as a separate bullet point.
    6. Avoid paragraphs; only bullet points should be used in the output.
<</SYS>>"""

p3 = "{SYS}\n Question: {PROMPT}. article: {BODY}\n Answer:".format(SYS=new_system_prompt, PROMPT=text_summarization_prompt, BODY=text)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer(p3, return_tensors='pt', padding=True, truncation=True).to(device)

outputs = model.generate(inputs['input_ids'],
                         attention_mask = inputs['attention_mask'],
                         max_new_tokens=300,
                         top_k=10,
                         num_return_sequences=1,
                         temperature=0.7,
                         repetition_penalty=1.0,
                         length_penalty=1.0,
                         eos_token_id=tokenizer.eos_token_id,
                         pad_token_id = tokenizer.pad_token_id,
                         early_stopping=True)

r3 = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
print(r3)

<<SYS>> You are a helpful assistant. 
    When summarizing an article or providing key points, you must:
    1. Use **bullet points** to list the main ideas.
    2. **Bullet points must be formatted like this:**
       - First key point goes here.
       - Second key point goes here.
       - Third key point goes here.
    3. Each bullet point should be one short sentence.
    4. Keep bullet points clear, concise, and easy to read.
    5. Make sure every distinct idea from the article is listed as a separate bullet point.
    6. Avoid paragraphs; only bullet points should be used in the output.
<</SYS>>
 Question:  Write a concise summary of the main ideas in article below.. article: Associazione Calcio Milan, commonly referred to as AC Milan or simply Milan, is an Italian 
professional football club based in Milan, Lombardy. Founded in 1899, the club competes in the Serie 
A, the top tier of Italian football. In its early history, Milan played its home games in different grounds 
arou

### Chain-of-Thought prompting

Chain-of-thought is when a prompt is being constructed using a previous prompt answer. For our use case to extract information from text, we will first ask Llama what the article is about and then use the response to ask a second question: what problem does [what the article is about] solve?



In [ ]:


r4_answer = r3.split('Answer:', 1)[1].strip()
#now embed the result of the previous prompt in a new prompt to ask what that solves


#define a prompt to ask what the article is about
additional_info_prompt = """How many titles did AC Milan win?"""
new_system_prompt = """<<SYS>> You are a helpful, respectful and honest assistant. \
    Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, \
    unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses \
    are socially unbiased and positive in nature. If a question does not make any sense, or is not factually \
    coherent, explain why instead of answering something not correct. If you don't know the answer to a question, \
    please don't share false information.
<</SYS>>"""
p5 = "{SYS}\n Question: {PROMPT}. article: {BODY}\n Answer:".format(SYS=new_system_prompt, PROMPT=additional_info_prompt, BODY=r4_answer)

print(f'PROMPT\n: {p5}\n\n\n')
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id


inputs = tokenizer(p5, return_tensors='pt', padding=True, truncation=True).to(device)

outputs = model.generate(inputs['input_ids'],
                         attention_mask = inputs['attention_mask'],
                         max_new_tokens=300,
                         max_length=500,
                         top_k=10,
                         num_return_sequences=1,
                         temperature=0.7,
                         repetition_penalty=1.0,
                         length_penalty=1.0,
                         eos_token_id=tokenizer.eos_token_id,
                         pad_token_id = tokenizer.pad_token_id,
                         early_stopping=True)

r5 = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

print(r5)



Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


PROMPT
: <<SYS>> You are a helpful, respectful and honest assistant.     Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,     unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses     are socially unbiased and positive in nature. If a question does not make any sense, or is not factually     coherent, explain why instead of answering something not correct. If you don't know the answer to a question,     please don't share false information.
<</SYS>>
 Question: How many titles did AC Milan win?. article: The summary is the following:
 
1. The club is a professional football club based in Milan, Italy.
2. The club competes in the Italian top-flight Serie A.
3. The club was founded in 1899 and has a long history in the sport.
4. Milan plays its home games at the San Siro stadium, which is the largest stadium in Italy.
5. The club has a rivalry with Inter Milan, with whom they compete in the 

### Generating JSONs with Llama

Llama needs precise instructions when asking it to generate JSON. In essence, here is what works for me to get valid JSON consistently:

- Explicitly state — “ All output must be in valid JSON. Don’t add explanation beyond the JSON” in the system prompt.
- Add an “explanation” variable to the JSON example. Llama enjoys explaining its answers. Give it an outlet.
- Use the JSON as part of the instruction. See the “in_less_than_ten_words” example below.
Change “write the answer” to “output the answer.”


In [ ]:


#example addition to a prompt to deal with jsons
json_prompt_addition = "Output must be in valid JSON like the following example {\"topic\": topic, \"explanation\": [in_less_than_ten_words]}. Output must include only JSON."


#define a prompt to ask what the article is about
new_system_prompt = """<<SYS>> You are a helpful, respectful and honest assistant. \
    Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, \
    unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses \
    are socially unbiased and positive in nature. If a question does not make any sense, or is not factually \
    coherent, explain why instead of answering something not correct. If you don't know the answer to a question, \
    please don't share false information.
    {ADDITION} <</SYS>>""".format(ADDITION=json_prompt_addition)

text_summarization_prompt3 = "Summarize the article below. Provide the different main aspects of AC Milan in JSON format in the format of the above example. Provide many different aspects"

p6 = "{SYS}\n Question: {PROMPT}. article: {BODY}\n Answer:".format(SYS=new_system_prompt, PROMPT=text_summarization_prompt3, BODY=text)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id


inputs = tokenizer(p6, return_tensors='pt', padding=True, truncation=True).to(device)

outputs = model.generate(inputs['input_ids'],
                         attention_mask = inputs['attention_mask'],
                         max_new_tokens=300,
                         max_length=500,
                         top_k=10,
                         num_return_sequences=1,
                         temperature=0.7,
                         repetition_penalty=1.0,
                         length_penalty=1.0,
                         eos_token_id=tokenizer.eos_token_id,
                         pad_token_id = tokenizer.pad_token_id,
                         early_stopping=True)

r6 = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

print(r6)

#compare the difference between the prompt with the formatting instruction and a regular prompt without formatting instructions. is there any difference?




Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<<SYS>> You are a helpful, respectful and honest assistant.     Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,     unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses     are socially unbiased and positive in nature. If a question does not make any sense, or is not factually     coherent, explain why instead of answering something not correct. If you don't know the answer to a question,     please don't share false information.
    Output must be in valid JSON like the following example {"topic": topic, "explanation": [in_less_than_ten_words]}. Output must include only JSON. <</SYS>>
 Question: Summarize the article below. Provide the different main aspects of AC Milan in JSON format in the format of the above example. Provide many different aspects. article: Associazione Calcio Milan, commonly referred to as AC Milan or simply Milan, is an Italian 
professional football club based in

### One-to-Many Shot Learning Prompting

One-to-Many Shot Learning is a term that refers to a type of machine learning problem where the goal is to learn to recognize many different classes of objects from only one or a few examples of each class. For example, if you have only one image of a cat and one image of a dog, can you train a model to distinguish between cats and dogs in new images? This is a challenging problem because the model has to generalize well from minimal data (source)

Important points about the prompts:

- The system prompt includes the instructions to output the answer in JSON.
- The prompt consists of an one-to-many shot learning section that starts after ```<</SYS>>``` and ends with ```</s>```.  See the prompt template below will make it easier to understand.
- The examples are given in JSON because the answers need to be JSON.
- The JSON allows defining the response with name, type, and explanation.
- The prompt question start with the second ```<s>[INST]``` and end with the last ```[/INST]```

```
<s>[INST] <<SYS>>
SYSTEM MESSAGE
<</SYS>>
EXAMPLE QUESTION [/INST]
EXAMPLE ANSWER(S)
</s>
<s>[INST]  
QUESTION
[/INST]
```

In [ ]:
#describe all the main nouns in the example.pdf article

#use the following addition for one-to-many prompting exampling
nouns = """[\
{{"name": "semiconductor", "type": "industry", "explanation": "Companies engaged in the design and fabrication of semiconductors and semiconductor devices"}},\
{{"name": "NBA", "type": "sport league", "explanation": "NBA is the national basketball league"}},\
{{"name": "Ford F150", "type": "vehicle", "explanation": "Article talks about the Ford F150 truck"}},\
{{"name": "Ford", "type": "company", "explanation": "Ford is a company that built vehicles"}},\
{{"name": "John Smith", "type": "person", "explanation": "Mentioned in the article"}},\
]"""

#now build the prompt following the template described above

#example addition to a prompt to deal with jsons
json_prompt_addition = "Output must only be in the following format, by describing names and descriptions exctracted by the article = {EXAMPLES}".format(EXAMPLES=nouns)


#define a prompt to ask what the article is about
new_system_prompt = """<<SYS>> You are a helpful, respectful and honest assistant. \
    Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, \
    unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses \
    are socially unbiased and positive in nature. If a question does not make any sense, or is not factually \
    coherent, explain why instead of answering something not correct. If you don't know the answer to a question, \
    please don't share false information.
    {ADDITION}.
    Output must include only this. 
    <</SYS>>""".format(ADDITION=json_prompt_addition)

text_summarization_prompt3 = "Summarize the article below. Provide the different main aspects of AC Milan in JSON format in the format of the above example. Provide many different info about Milan"

p7 = "{SYS}\n Question: {PROMPT}. article: {BODY}\n Answer:".format(SYS=new_system_prompt, PROMPT=text_summarization_prompt3, BODY=text)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id


inputs = tokenizer(p7, return_tensors='pt', padding=True, truncation=True).to(device)

outputs = model.generate(inputs['input_ids'],
                         attention_mask = inputs['attention_mask'],
                         max_new_tokens=300,
                         max_length=500,
                         top_k=10,
                         num_return_sequences=1,
                         temperature=0.7,
                         repetition_penalty=1.0,
                         length_penalty=1.0,
                         eos_token_id=tokenizer.eos_token_id,
                         pad_token_id = tokenizer.pad_token_id,
                         early_stopping=True)

r7 = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

print(r7)
#compare the response of the prompt described above and a zero-shot prompt. Are there any differences?


Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<<SYS>> You are a helpful, respectful and honest assistant.     Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,     unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses     are socially unbiased and positive in nature. If a question does not make any sense, or is not factually     coherent, explain why instead of answering something not correct. If you don't know the answer to a question,     please don't share false information.
    Output must only be in the following format, by describing names and descriptions exctracted by the article = [{{"name": "semiconductor", "type": "industry", "explanation": "Companies engaged in the design and fabrication of semiconductors and semiconductor devices"}},{{"name": "NBA", "type": "sport league", "explanation": "NBA is the national basketball league"}},{{"name": "Ford F150", "type": "vehicle", "explanation": "Article talks about the Ford F150 tr

## Exercise 2: RAG (Retrieval-Augmented-Generation)

RAG (Retrieval-Augmented Generation) is a powerful framework in Natural Language Processing (NLP) that enhances the performance of language models by combining traditional generative models with external knowledge retrieval. This hybrid approach allows models to retrieve relevant information from a large corpus (like a database or document collection) and incorporate this information into the generation process. It is particularly useful when a model needs to answer questions, generate content, or provide explanations based on real-time or domain-specific data.



In [51]:
import os
import glob


def extract_text_from_pdf(pdf_path):
    return read_pdf(fitz.open(pdf_path))
    
# Extract text from all uploaded PDF files
pdf_texts = {f'paper{i}': extract_text_from_pdf(f'paper{i}.pdf') for i in range(10)}

#Display the text from all the PDF files
for pdf_file, text in pdf_texts.items(): 
    print(f"--- {pdf_file} ---")
    print(text[:500])  # Display the first 500 characters of each document
    print("\n") 

--- paper0 ---
XIWU: A BASIS FLEXIBLE AND LEARNABLE LLM FOR HIGH
ENERGY PHYSICS
Zhengde Zhang1, Yiyu Zhang1, Haodong Yao1, Jianwen Luo2, Rui Zhao1, Bo Huang1, Jiameng Zhao3,1,
Yipu Liao1, Ke Li1, Lina Zhao1, Jun Cao1, Fazhi Qi1,∗, and Changzheng Yuan1,†
1Institute of High Energy Physics, Chinese Academy of Sciences, Beijing 100049, China
2School of Physical Sciences, University of Chinese Academy of Sciences, Beijing 100049, China
3School of Computer and Artificial Intelligence, ZhengZhou University, Henan 450


--- paper1 ---
1
Towards Greener LLMs: Bringing Energy-Efficiency
to the Forefront of LLM Inference
Jovan Stojkovic, Esha Choukse†, Chaojie Zhang†, Inigo Goiri†, Josep Torrellas
University of Illinois at Urbana-Champaign
†Microsoft Azure Research - Systems
Abstract—With the ubiquitous use of modern large language
models (LLMs) across industries, the inference serving for these
models is ever expanding. Given the high compute and memory
requirements of modern LLMs, more and more

### Creating an index of vectors to represent the documents

To perform efficient searches, we need to convert our text data into numerical vectors. To do so, we will use the first step of the BERT transformer.

Since our full pdf files are very long to be fed as input into BERT, we perform a step in which we create a structure where we associate a document number to its abstract, and in a separate dictionary we associate a document number to its full text.


In [57]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [56]:
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np




#import the Bert pretrained model from the transformers library
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model.to(device)

#initialization of the dictionary of abstracts. Substitute this with the abstracts of the 10 papers considered as sources for RAG
#(we could use functions to read the PDFs to "cut" the abstracts from the papers. For simplicity reasons, we will copy and paste them)
abstracts_dict = {
    0: """Large Language Models (LLMs) are undergoing a period of rapid updates and changes, with state-
of-the-art (SOTA) model frequently being replaced. When applying LLMs to a specific scientific
field, it’s challenging to acquire unique domain knowledge while keeping the model itself advanced.
To address this challenge, a sophisticated large language model system named as Xiwu has been
developed, allowing you switch between the most advanced foundation models and quickly teach the
model domain knowledge. In this work, we will report on the best practices for applying LLMs in the
field of high-energy physics (HEP), including: a seed fission technology is proposed and some data
collection and cleaning tools are developed to quickly obtain domain AI-Ready dataset; a just-in-time
learning system is implemented based on the vector store technology; an on-the-fly fine-tuning system
has been developed to facilitate rapid training under a specified foundation model.
The results show that Xiwu can smoothly switch between foundation models such as LLaMA, Vicuna,
ChatGLM and Grok-1. The trained Xiwu model is significantly outperformed the benchmark model
on the HEP knowledge Q&A and code generation. This strategy significantly enhances the potential
for growth of our model’s performance, with the hope of surpassing GPT-4 as it evolves with the
development of open-source models. This work provides a customized LLM for the field of HEP,
while also offering references for applying LLM to other fields, the corresponding codes are available
on Github https://github.comzhang/zhengde0225/Xiwu.""",
    1: """
Abstract—With the ubiquitous use of modern large language
models (LLMs) across industries, the inference serving for these
models is ever expanding. Given the high compute and memory
requirements of modern LLMs, more and more top-of-the-
line GPUs are being deployed to serve these models. Energy
availability has come to the forefront as the biggest challenge for
data center expansion to serve these models. In this paper, we
present the trade-offs brought up by making energy efficiency
the primary goal of LLM serving under performance SLOs.
We show that depending on the inputs, the model, and the
service-level agreements, there are several knobs available to
the LLM inference provider to use for being energy efficient.
We characterize the impact of these knobs on the latency,
throughput, as well as the energy. By exploring these trade-
offs, we offer valuable insights into optimizing energy usage
without compromising on performance, thereby paving the way
for sustainable and cost-effective LLM deployment in data center
environments
    """,
    2: """
The rapid adoption of large language models (LLMs) has led to
significant advances in natural language processing and text gener-
ation. However, the energy consumed through LLM model infer-
ence remains a major challenge for sustainable AI deployment. To
address this problem, we model the workload-dependent energy
consumption and runtime of LLM inference tasks on heterogeneous
GPU-CPU systems. By conducting an extensive characterization
study of several state-of-the-art LLMs and analyzing their energy
and runtime behavior across different magnitudes of input prompts
and output text, we develop accurate (𝑅2 >0.96)energy and run-
time models for each LLM. We employ these models to explore
an offline, energy-optimal LLM workload scheduling framework.
Through a case study, we demonstrate the advantages of energy
and accuracy aware scheduling compared to existing best practices
    """,
    3: """
The growing demand for efficient and scalable AI solutions has driven research into optimizing the performance and energy
efficiency of computational infrastructures. The novel concept of redesigning inference clusters and modifying the GPT-Neo
model offers a significant advancement in addressing the computational and environmental challenges associated with AI
deployment. By developing a novel cluster architecture and implementing strategic architectural and algorithmic changes,
the research achieved substantial improvements in throughput, latency, and energy consumption. The integration of advanced
interconnect technologies, high-bandwidth memory modules, and energy-efficient power management techniques, alongside
software optimizations, enabled the redesigned clusters to outperform baseline models significantly. Empirical evaluations
demonstrated superior scalability, robustness, and environmental sustainability, emphasizing the potential for more sustainable
AI technologies. The findings underscore the importance of balancing performance with energy efficiency and provide a robust
framework for future research and development in AI optimization. The research contributes valuable insights into the design
and deployment of more efficient and environmentally responsible AI systems.
1
""",
    4: """
Establishing building energy models (BEMs) for building design and analysis poses significant challenges due to demanding modeling efforts, expertise to use simulation software, and building science knowledge in practice. These make building modeling labor-intensive, hindering its widespread adoptions in building development. Therefore, to overcome these challenges in building modeling with enhanced automation in modeling practice, this paper proposes Eplus-LLM (EnergyPlus-Large Language Model) as the auto-building modeling platform, building on a fine-tuned large language model (LLM) to directly translate natural language description of buildings to established building models of various geometries, occupancy scenarios, and equipment loads. Through fine-tuning, the LLM (i.e., T5) is customized to digest natural language and simulation demands from users and convert human descriptions into EnergyPlus modeling files. Then, the Eplus-LLM platform realizes the automated building modeling through invoking the API of simulation software (i.e., the EnergyPlus engine) to simulate the auto-generated model files and output simulation results of interest. The validation process, involving four different types of prompts, demonstrates that Eplus-LLM reduces over 95% modeling efforts and achieves 100% accuracy in establishing BEMs while being robust to interference in usage, including but not limited to different tones, misspells, omissions, and redundancies. Overall, this research serves as the pioneering effort to customize LLM for auto-modeling purpose (directly build-up building models from natural language), aiming to provide a user-friendly human-AI interface that significantly reduces building modeling efforts. This work also further facilitates large-scale building model efforts, e.g., urban building energy modeling (UBEM), in modeling practice.
""",
5: """
The rapid evolution and widespread adoption of
generative large language models (LLMs) have made them a
pivotal workload in various applications. Today, LLM inference
clusters receive a large number of queries with strict Service
Level Objectives (SLOs). To achieve the desired performance,
these models execute on power-hungry GPUs causing the in-
ference clusters to consume large amount of energy and, conse-
quently, result in excessive carbon emissions. Fortunately, we find
that there is a great opportunity to exploit the heterogeneity in
inference compute properties and fluctuations in inference work-
loads, to significantly improve energy-efficiency. However, such
a diverse and dynamic environment creates a large search-space
where different system configurations (e.g., number of instances,
model parallelism, and GPU frequency) translate into different
energy-performance trade-offs. To address these challenges, we
propose DynamoLLM, the first energy-management framework
for LLM inference environments. DynamoLLM automatically
and dynamically reconfigures the inference cluster to optimize for
energy and cost of LLM serving under the service’s performance
SLOs. We show that at a service-level, DynamoLLM conserves
53 percent energy and 38 percent operational carbon emissions, and reduces
61 percent cost to the customer, while meeting the latency SLOs
""",
6: """
Large language model (LLM) has recently been
considered a promising technique for many fields. This work
explores LLM-based wireless network optimization via in-context
learning. To showcase the potential of LLM technologies, we
consider the base station (BS) power control as a case study,
a fundamental but crucial technique that is widely investigated
in wireless networks. Different from existing machine learning
(ML) methods, our proposed in-context learning algorithm relies
on LLM’s inference capabilities. It avoids the complexity of
tedious model training and hyper-parameter fine-tuning, which is
a well-known bottleneck of many ML algorithms. Specifically, the
proposed algorithm first describes the target task via formatted
natural language, and then designs the in-context learning
framework and demonstration examples. After that, it considers
two cases, namely discrete-state and continuous-state problems,
and proposes state-based and ranking-based methods to select
appropriate examples for these two cases, respectively. Finally, the
simulations demonstrate that the proposed algorithm can achieve
comparable performance as conventional deep reinforcement
learning (DRL) techniques without dedicated model training or
fine-tuning. Such an efficient and low-complexity approach has
great potential for future wireless network optimization.
Index Terms—Large language model, in-context learning, net-
work optimization, transmission power contro
""",
7: """
Both the training and use of Large Language Models (LLMs) require
large amounts of energy. Their increasing popularity, therefore,
raises critical concerns regarding the energy efficiency and sus-
tainability of data centers that host them. This paper addresses the
challenge of reducing energy consumption in data centers running
LLMs. We propose a hybrid data center model that uses a cost-based
scheduling framework to dynamically allocate LLM tasks across
hardware accelerators that differ in their energy efficiencies and
computational capabilities. Specifically, our workload-aware strat-
egy determines whether tasks are processed on energy-efficient
processors or high-performance GPUs based on the number of in-
put and output tokens in a query. Our analysis of a representative
LLM dataset, finds that this hybrid strategy can reduce CPU+GPU
energy consumption by 7.5 percent compared to a workload-unaware
baseline
""",
8: """
Reproducible science requires easy access to data, especially with
the rise of data-driven and increasingly complex models used within
energy research. Too often however, the data to reconstruct and
verify purported solutions in publications is hidden due to some
combination of commercial, legal, and sensitivity issues. This early
work presents our initial efforts to leverage the recent advance-
ments in Large Language Models (LLMs) to create usable and share-
able energy datasets. In particular, we’re utilising their mimicry of
human behaviors, with the goal of extracting and exploring syn-
thetic energy data through the simulation of LLM agents capable of
interacting with and executing actions in controlled environments.
We also analyse and visualise publicly available data in an attempt
to create realistic but not quite exact copies of the originals. Our
early results show some promise, with outputs that resemble the
twin peak curves for household energy consumption. The hope is
that our generalised approach can be used to easily replicate usable
and realistic copies of otherwise secret or sensitive data
""",
9: """
This paper introduces a method for personaliz-
ing energy optimization using large language models (LLMs)
combined with an optimization solver. This approach, termed
human-guided optimization autoformalism, translates natural
language specifications into optimization problems, enabling
LLMs to handle various user-specific energy-related tasks. It
allows for nuanced understanding and nonlinear reasoning
tailored to individual preferences. The research covers common
energy sector tasks like electric vehicle charging, HVAC control,
and long-term planning for renewable energy installations. This
novel strategy represents a significant advancement in context-
based optimization using LLMs, facilitating sustainable energy
practices customized to individual needs
"""
}

#the text for rag is used as an input to the BERT model

#The tokenized inputs are passed to the BERT model for processing.
#(#remember padding=True: Ensures that all inputs are padded to the same length, allowing batch processing.)
#The model outputs a tensor (last_hidden_state), where each input token is represented by a high-dimensional vector.
#last_hidden_state is of shape (batch_size, sequence_length, hidden_size), where:
#batch_size: Number of input texts.
#sequence_length: Length of each tokenized text (after padding).
#hidden_size: Dimensionality of the vector representation for each token (default 768 for bert-base-uncased).

#last_hidden_state[:, 0]: Selects the representation of the [CLS] token for each input text. The [CLS] token is a special token added at the start of each input and is often used as the aggregate representation for the entire sequence.
tokenized_inputs = tokenizer(list(abstracts_dict.values()), return_tensors='pt', padding=True)
#same but with values on GPU
tokenized_inputs = {key: value.to(device) for key, value in tokenized_inputs.items()}

print(model(**tokenized_inputs))
abstract_vectors = model(**tokenized_inputs).last_hidden_state[:, 0]

#abstract_vectors is a tensor of shape (batch_size, hidden_size) (e.g., (3, 768) in this case), representing each text as a single 768-dimensional vector (CLS).

print(abstract_vectors.shape)



BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3220, -0.3335, -0.1954,  ..., -0.2925,  0.2846,  0.3641],
         [-0.5645,  0.4611,  0.7577,  ..., -0.2724,  0.3017, -0.3110],
         [-0.9413, -0.0867,  1.0387,  ..., -0.2631,  0.0400,  0.5614],
         ...,
         [-0.1484, -0.2075, -0.1752,  ...,  0.0926, -0.1863,  0.1050],
         [-0.2090, -0.1377, -0.0613,  ..., -0.0501, -0.2226,  0.1268],
         [-0.0029, -0.0475, -0.0555,  ..., -0.1070, -0.2341,  0.2976]],

        [[-0.5559, -0.2410, -0.5079,  ..., -0.3529,  0.4078, -0.1399],
         [ 0.2008,  0.7545, -0.3053,  ...,  0.0627,  0.5156, -0.0106],
         [ 0.0417,  0.2561,  0.0759,  ..., -0.6539,  0.1648,  0.0106],
         ...,
         [-0.3017, -0.4531,  0.0994,  ...,  0.0924,  0.0874, -0.8269],
         [ 0.5455,  0.6252,  0.0647,  ..., -0.2675, -0.0738, -0.1672],
         [ 0.2909,  0.0319,  0.4529,  ..., -0.0058, -0.0636, -0.6091]],

        [[-0.4784, -0.1242, -0.2999,  ..., -0.3500,  

### Search

With our text data vectorized and indexed, we can now perform searches. We will define a function to search the index for the most relevant documents based on a query.

To perform the search, we need a function (search documents) where we perform the cosine similarity between the query vector and all the abstract vectors. This function will give our the top-k indexes. Once we find the top-k indexes, with another function, we can collect the full text of the documents from the paper dictionary.

To compute cosine similarity, refer to the following formula

```cs = cosine_similarity(vector_a.cpu().detach().numpy(), vector_b.cpu().detach().numpy())```



In [76]:


def get_top_k_similar_indices(query_vector, abstract_vectors, k):
    
    #Computes the top k indices of the most similar abstracts to the query based on cosine similarity.
    
    #Parameters:
    #- query_vector: A tensor of shape (1, hidden_size) representing the query vector.
    #- abstract_vectors: A tensor of shape (batch_size, hidden_size) representing the abstract vectors.
    #- k: The number of top indices to return.
    
    #Returns:
    #- sorted_indices: A numpy array of shape (1, k) containing the indices of the top k most similar abstracts.
    cosines = cosine_similarity(query_vector.cpu().detach().numpy(), abstract_vectors.cpu().detach().numpy())
    sorted_indices = np.argsort(cosines, axis=1)[:, ::-1] #inverse

    return sorted_indices[:, :k]


def retrieve_documents(indices, documents_dict):
    
    #Retrieves the documents corresponding to the given indices and concatenates them into a single string.
    
    #Parameters:
    #- indices: A numpy array or list of top-k indices of the most similar documents.
    #- documents_dict: A dictionary where keys are document indices (integers) and values are the document texts (strings).
    
    #Returns:
    #- concatenated_documents: A string containing the concatenated texts of the retrieved documents.


    text_list = [documents_dict[f'paper{i}'] for i in indices[0]]
    text= ' '.join(text_list)

    return text



#now I create a vector also for my query 

query = "energy topics covered by papers with large language models"

inputs = tokenizer(query, return_tensors='pt', padding=True)
inputs = {key: value.to(device) for key, value in inputs.items()}

query_vector = model(**inputs).last_hidden_state[:, 0]

print(f"Model is on device: {model.device}")
print(f"Query tensor is on device: {inputs['input_ids'].device}")

indices = get_top_k_similar_indices(query_vector, abstract_vectors, 1)

print(f"BEST DOCUMENTS: {indices}")
concatenated_documents=retrieve_documents(indices, pdf_texts)
print(concatenated_documents)




Model is on device: cuda:0
Query tensor is on device: cuda:0
BEST DOCUMENTS: [[9]]
Democratizing Energy Management with LLM-Assisted Optimization
Autoformalism
Ming Jin, Bilgehan Sel, Fnu Hardeep, Wotao Yin
Abstract— This paper introduces a method for personaliz-
ing energy optimization using large language models (LLMs)
combined with an optimization solver. This approach, termed
human-guided optimization autoformalism, translates natural
language speciﬁcations into optimization problems, enabling
LLMs to handle various user-speciﬁc energy-related tasks. It
allows for nuanced understanding and nonlinear reasoning
tailored to individual preferences. The research covers common
energy sector tasks like electric vehicle charging, HVAC control,
and long-term planning for renewable energy installations. This
novel strategy represents a signiﬁcant advancement in context-
based optimization using LLMs, facilitating sustainable energy
practices customized to individual needs.
I. INTRODUCTION
De

### A function to perform Retrieval Augmented Generation

In this step, we’ll combine the context retrieved from our documents with LLAMA to generate responses. The context will provide the necessary information to the model to produce more accurate and relevant answers.

In [95]:
model_id = "meta-llama/Llama-3.2-3B"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id)
device = "cuda"
model.to(device)

if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

#now we put it all together

def generate_augmented_response(query, documents):
    modelBERT = AutoModel.from_pretrained("bert-base-uncased")
    tokenizerBERT = AutoTokenizer.from_pretrained("bert-base-uncased")
    modelBERT.to(device)


    if tokenizerBERT.pad_token_id is None:
        tokenizerBERT.pad_token_id = tokenizerBERT.eos_token_id

    system = """<<SYS>> You are a helpful, respectful and honest assistant. \
    Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, \
    unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses \
    are socially unbiased and positive in nature. If a question does not make any sense, or is not factually \
    coherent, explain why instead of answering something not correct. If you don't know the answer to a question, \
    please don't share false information.
    Provide straight to the point answers, without bullet points.
<</SYS>>"""

    query_for_rag = query    
    inputs = tokenizerBERT(query, return_tensors='pt', padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    query_vector = modelBERT(**inputs).last_hidden_state[:, 0]
    indices = get_top_k_similar_indices(query_vector, abstract_vectors, 1)



    context = retrieve_documents(indices, documents)[:6000] #the context is the entire document that matches the query

    
    prompt =  """{SYS}\nQuestion: {PROMPT}. Collect information from the following article: {BODY} \nAnswer:""".format(SYS=system, PROMPT=query_for_rag, BODY=context)

    
    


    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(device)

    outputs = model.generate(inputs['input_ids'],
                            attention_mask = inputs['attention_mask'],
                            max_new_tokens=300,
                            max_length=500,
                            top_k=10,
                            num_return_sequences=1,
                            temperature=0.7,
                            repetition_penalty=1.0,
                            length_penalty=1.0,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id = tokenizer.pad_token_id,
                            early_stopping=True)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    return response


query = "What are the main energy issues for Large Language Models?"
response = generate_augmented_response(query, pdf_texts)
print(response)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.92s/it]
Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<<SYS>> You are a helpful, respectful and honest assistant.     Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,     unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses     are socially unbiased and positive in nature. If a question does not make any sense, or is not factually     coherent, explain why instead of answering something not correct. If you don't know the answer to a question,     please don't share false information.
    Provide straight to the point answers, without bullet points.
<</SYS>>
Question: What are the main energy issues for Large Language Models?. Collect information from the following article: Offline Energy-Optimal LLM Serving: Workload-Based Energy
Models for LLM Inference on Heterogeneous Systems
Grant Wilkins
gfw27@cam.ac.uk
University of Cambridge
Cambridge, UK
Srinivasan Keshav
sk818@cam.ac.uk
University of Cambridge
Cambridge, UK
Richard Mortier
rmm1002@